# Paso #1: IMPORTAR LIBRERÍAS

In [ ]:
!wget https://www.dropbox.com/s/i4n134sqzu7psry/fotos%20procesadas.zip?dl=0

In [ ]:
!wget https://www.dropbox.com/s/u5fu6o8285ebsne/Fotos%20TFM.zip?dl=0

In [ ]:
!unzip Fotos\ TFM.zip\?dl\=0

In [ ]:
!unzip fotos\ procesadas.zip\?dl\=0

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Funciones y metodos reutilizables
def elem2dict(node):
    """
    Convert an xml.ElementTree node tree into a dict.
    """
    result = {}

    for element in node:
        key = element.tag
        if '}' in key:
            # Remove namespace prefix
            key = key.split('}')[1]

        if node.attrib:
            result['@attribs'] = dict(node.items())

        # Process element as tree element if the inner XML contains non-whitespace content
        if element.text and element.text.strip():
            value = element.text
        else:
            value = elem2dict(element)

        # Check if a node with this name at this depth was already found
        if key in result:
            if type(result[key]) is not list:
                # We've seen it before, but only once, we need to convert it to a list
                tempvalue = result[key].copy()
                result[key] = [tempvalue, value]
            else:
                # We've seen it at least once, it's already a list, just append the node's inner XML
                result[key].append(value)
        else:
            # First time we've seen it
            result[key] = value

    return result

def get_data_imgLabel(imgLabel):
    x = imgLabel["bndbox"]["xmin"]
    y = imgLabel["bndbox"]["ymin"]
    width = imgLabel["bndbox"]["xmax"]
    height = imgLabel["bndbox"]["ymax"]

    return x,y, width, height

# Funcion para redimensionar la imagen  a (512, 512)
# para redimensionar la imagen  a (512, 512)
def resize(x):
  return cv2.resize(x, dsize=(widthImage, heightImage), interpolation = cv2.INTER_AREA)

In [ ]:
# Importamos los paquetes necesarios
import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import json
from xml.etree import ElementTree as ET
import random
from PIL import Image

# Ruta de la data y capertas dentro de esta
dataPath = '/content/Fotos TFM'
dataPathXML = '/content/fotos procesadas'
typeLeavesList = os.listdir(dataPathXML)

widthImage = 224
heightImage = 224
quantitytoProcess = 1000
counterSubPlot = 1

# data
labels = []
leavesData = []
labelCounter = 0

# Leemos la información de los Json en las carpetas, para obtener las coordenadas de las imagenes etiquetadas.
listCount = []
dataToProcess = []

for nameDir in typeLeavesList:
    leavePathXML = dataPathXML + '/' + nameDir
    leavePathImg = dataPath + '/' + nameDir

    if "Sanas" == nameDir:
      labelCounter = 0

    if "Enfermas" == nameDir:
      labelCounter = 1

    # lista de archivos .json con las etiquetas de las imagenes
    listXML =  [f for f in os.listdir(leavePathXML) if f.endswith('.xml')]

    listXML = listXML[:quantitytoProcess]

    countFilesXML = len([name for name in listXML ])
    print('Cantidad de imágenes:', nameDir)
    print(countFilesXML)

    sample20ListXML = random.sample(listXML, 20)

    # Cantidad de archivos a procesar
    listCount.append(countFilesXML)

    for fileXML in listXML:

        if len(list(filter(lambda x: x["label"] == labelCounter, dataToProcess))) > quantitytoProcess:
            continue

        xmlParsed = ET.parse(leavePathXML + '/'+ fileXML).getroot()
        xmlDict = elem2dict(xmlParsed)
        imageName = xmlDict["filename"]

        if "object" not in xmlDict:
            continue

        objectImgLabel = xmlDict["object"]

        # Obtenemos la imagen completa
        image = cv2.imread(leavePathImg+'/'+ imageName)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if (type(objectImgLabel) == list):
            for imgLabel in objectImgLabel:
                x, y, width, height = get_data_imgLabel(imgLabel)
                roi = image[int(y):int(height), int(x):int(width)]

                leafData = {
                    "filenameImg": imageName,
                    "filenameXML": fileXML,
                    "label" : labelCounter,
                    "roi": resize(roi)
                }
                dataToProcess.append(leafData)

        else:
            x, y, width, height = get_data_imgLabel(objectImgLabel)
            roi = image[int(y):int(height), int(x):int(width)]
            leafData = {
                "filenameImg": imageName,
                "filenameXML": fileXML,
                "label" : labelCounter,
                "roi": resize(roi)
            }
            dataToProcess.append(leafData)


newListEnfermas = filter(lambda x: x["label"] == 1, dataToProcess)
newListSanas = filter(lambda x: x["label"] == 0, dataToProcess)

listEnfermas = list(newListEnfermas)
listSanas = list(newListSanas)


In [ ]:
# # Mostrar las primeras 40 imágenes enfermas
fig = plt.figure(figsize=(15, 15))

for i in range(40):
    image = listEnfermas[i]["roi"]
    ax = fig.add_subplot(5, 8, i + 1)
    ax.imshow(image)
    ax.axis("off")

plt.show()

In [ ]:
# # Mostrar las primeras 40 imágenes SANAS
fig = plt.figure(figsize=(15, 15))

for i in range(40):
    image = listSanas[i]["roi"]
    ax = fig.add_subplot(5, 8, i + 1)
    ax.imshow(image)
    ax.axis("off")

plt.show()

In [ ]:
# Visualiacion de cantidad de imagenes por clasificación - Enfermas / Sanas
plt.figure(figsize = (5,5))
sns.barplot(x = typeLeavesList, y = listCount )

In [ ]:
print('Número de etiquetas 1 (Enfermas): ', len(listEnfermas))
print('Número de etiquetas 0(Sanas): ', len(listSanas))

# Paso #3: PREPROCESADOR DE LAS IMÁGENES

In [ ]:
# Dividir el dataframe en características y etiquetas para la clasificación
from keras.utils import to_categorical

X = np.array([x["roi"] for x in dataToProcess])
y = to_categorical([x["label"] for x in dataToProcess] )

# # Visualizamos los pixeles de las imagenes
X[0]

In [ ]:
# Visualizamos las etiquetas
y

In [ ]:
# Revisando la dimensionalidad de X
X.shape

In [ ]:
X = np.stack(X, axis = 0)
# aqui era uno, pero algo ha pasado y se cambio a 3
X = X.reshape(len(listEnfermas)+len(listSanas) , 224, 224, 3)

print(X.shape, y.shape)

In [ ]:
# Dividir el dataframe en conjunto de entrenamiento, test y validación

from sklearn.model_selection import train_test_split

# Obtenemos el conjunto de entrenamiento y el conjunto de Test
X_train, X_Test, y_train, y_Test = train_test_split(X, y, test_size = 0.4, shuffle = True)

# Del conjunto de Test, obtenemos una parte para el conjunto de validaciòn, el 50% del conjunto de Test
X_val, X_Test, y_val, y_Test = train_test_split(X_Test, y_Test, test_size = 0.6, shuffle = True)

In [ ]:
# Limpiar memoria
del dataToProcess
del X
del y


In [ ]:
print("Conjunto de validación")
print(X_val.shape, y_val.shape)
print("Conjunto de Test")
print(X_Test.shape, y_Test.shape)
print("Conjunto de Entrenamiento")
print(X_train.shape, y_train.shape)

# Paso #4: NORMALIZACIÓN DE LOS DATOS Y PREPARACIÓN PARA EL ENTRENAMIENTO

# Paso #3: AUMENTACIÓN DE LAS IMÁGENES

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # rotation_range=90,
    # width_shift_range=0.05,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # vertical_flip = True,
    brightness_range = [1.1, 1.15]
    )

valid_datagen = ImageDataGenerator(rescale=1./255)


test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # vertical_flip = True,
    brightness_range = [1.1, 1.15]
    )


batch_size = 16

valid_generator = valid_datagen.flow(
    X_val, y_val,
    batch_size=batch_size
)
train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=batch_size
)

test_generator = test_datagen.flow(
    X_Test, y_Test,
    batch_size=batch_size
)

In [ ]:
# Probando otro modelo
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

input_shape = (224, 224, 3)
num_classes = 2
batch_size = 16

base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
from time import gmtime, strftime
# Usamos la parada temprana para salir del entenamiento si el error de validación
# no decrece después de cierto número de epochs (paciencia)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)
fileTime = strftime("%Y-%m-%d %H:%M:%S", gmtime())
print(fileTime)
# Guardamos el mejor modelo con menor error de validación
checkpointer = ModelCheckpoint(
    filepath = "/content/drive/MyDrive/Master TFM/TFM/proyecto tfm/fotos procesadas/movilnet_weigth"+fileTime+".hdf5",
    verbose = 1,
    save_best_only=True,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max')

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=60, validation_data=valid_generator, callbacks=[checkpointer, earlystopping])


# TAREA #6: CONSTRUIR UN MODELO DE DEEP LEARNING(CNN) DE RESTNET(RED RESIDUAL)

In [ ]:
score = model.evaluate(X_Test, y_Test)
score_acurracy = score[1]
print('Accuracy en la fase de Test: {}'.format(score_acurracy))

In [ ]:
fileTime = strftime("%Y-%m-%d %H:%M:%S", gmtime())
print(fileTime)
model.save('/content/drive/MyDrive/Master TFM/TFM/proyecto tfm/fotos procesadas/mode_mobilenet '+str(score_acurracy)+ " --"+fileTime+'.h5')

In [ ]:
# Cargando Mejor Modelo Existente

model_clasification_leaves = load_model("/content/drive/MyDrive/Master TFM/TFM/proyecto tfm/fotos procesadas/leaf_model Accuracy 0.978935718536377 --2023-07-09 15:04:06.h5")

score = model_clasification_leaves.evaluate(X_Test, y_Test)
print('Accuracy en la fase de Test: {}'.format(score[1]))